In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
import time
import functools
from collections import Counter
#from nxviz import MatrixPlot
from datetime import datetime 
import matplotlib.dates as mdates
import pdb
import matplotlib.ticker as mticker
import pickle
from NetDistr import NetDistr
import seaborn as sb
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec
from matplotlib import rc
from tqdm import tqdm
import sys
import os
from PlotNet2 import PlotNet

In [2]:
# comp ='n'
# year = '2015'
# num_stations = 128
# dt_start = '2015-03-17 4:00:00'
# dt_end = '2015-03-17 12:00:00'
# ulf_fname = 'networks_data/ulf_spd_2015.csv'
# indices_fname = 'networks_data/indices_spd_2015.csv'
# net_labs = ['dir_net_t1', 'dir_net_tn', 'undir_net_inphase', 'undir_net_antiphase']
# path = f'networks_data/{year}_nets/comp_{comp}/'

# pn = PlotNet(net_labs, path, ulf_fname, num_stations, dt_start, dt_end)
# num_conn = pn.combined_nets_conn(save_analysis_files=False)


In [4]:
year = '2015'
num_stations = 128
dt_start ='2015-03-17 19:30:00'
dt_end='2015-03-18 03:30:00'
ulf_fname = 'networks_data/spd_ulf_min_smr.csv'
indices_fname = 'networks_data/indices_spd_2015.csv'
net_labs = ['dir_net_t1', 'dir_net_tn', 'undir_net_inphase', 'undir_net_antiphase']
# for saving plots
handel = 'smr_min'
pn = PlotNet(net_labs, ulf_fname, num_stations, dt_start, dt_end)
# num_conn = pn.combined_nets_conn(save_analysis_files=True)

In [ ]:
# year = 2013
# num_stations = 122
# dt_start = '2013-03-17 4:00:00'
# dt_end = '2013-03-17 12:00:00'
# ulf_fname = 'networks_data/17march2013_4am.csv'
# indices_fname = 'networks_data/170313_indices.csv'
# pn = PlotNet(net_labs, path, ulf_fname, num_stations, dt_start, dt_end)
# num_conn = pn.combined_nets_conn(save_analysis_files=False)

In [ ]:
# year = 2012
# num_stations = 133
# dt_start = '2012-01-21 20:00:00'
# dt_end = '2012-01-22 16:00:00'
# path = f'networks_data/{year}_nets/comp_{comp}/'
# ulf_fname = 'networks_data/2012-01-21_22-ulf.csv'
# indices_fname = 'networks_data/2012-01-21_22_indices.csv'
# pn = PlotNet(net_labs, path, ulf_fname, num_stations, dt_start, dt_end)
# num_conn = pn.combined_nets_conn(save_analysis_files=False)

In [ ]:
def plot_edge_num_ts1(num_conn:dict, indices_fname:str):
    plt.rcParams.update({'font.size': 15})
    num_plots = 5
    fig, ax = plt.subplots(num_plots, figsize=(15, 10))
    fig.suptitle('spd')
    indices = pd.read_csv(indices_fname)
    index_date = indices['Date_UTC']
    sme = indices['SME']
    axsmer = pd.to_datetime(index_date)
    datemin = datetime.strptime(pn.dt_start, '%Y-%m-%d %H:%M:%S')
    datemax = datetime.strptime(pn.dt_end, '%Y-%m-%d %H:%M:%S')
    f1 = ax[0]
    f1.plot(axsmer,sme, color='black', label = 'SME')
    f1.set_ylabel('SME (nT)')
    f1.grid()
#   turn off labels for x axis while preserving ticks
    f1.set_xticklabels([])                           
    f1.set_xlim(datemin,datemax)
    formatter = mdates.DateFormatter("%H:%M")
    f1.xaxis.set_major_formatter(formatter)
    f1.legend()
    f1.set_xlim(datemin,datemax)
    ax2 = f1.twinx()

    # code for plotting SMR sections for entire SMR yearly dataset
    indices['Date_UTC'] = pd.to_datetime(indices['Date_UTC']) 
    for i in ['SMR00','SMR06','SMR12','SMR18' ]:
        x = axsmer[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)]
        y = indices[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)][i]
        ax2.plot(x , y, label= i)
    ax2.xaxis.set_major_formatter(formatter)
    ax2.legend(loc='upper center',ncol=4, bbox_to_anchor=[0.5, 1.4])
    ax2.set_ylabel('SMR (nT)')
    ax2.grid()
    ax2.set_xlim(datemin,datemax)
    ax2.set_xticklabels([])

    dyn_pressure = indices['PDYN']
    # GSM field values
    bz, bx, by = indices['GSM_Bz'], indices['GSM_Bx'], indices['GSM_By']
    f2 = ax[1]
    f3 = ax[2]
    for j in [0,1,2,3]:
        y_vals = [bx, by, bz, dyn_pressure]
        y_labs = ['GSM_Bz','GSM_Bx','GSM_By','Dynamic pressure']
        c = ['red','black','green','orange']
        if j == 3:
            f3.set_ylabel('nPa')
            # get rid of peaks
            y = np.where( y_vals[j] == np.max(y_vals[j]) , np.nan , y_vals[j])
            f3.plot(axsmer, y, label= y_labs[j], color = 'green')
            f = f3
        else:
            y = np.where( y_vals[j] == np.max(y_vals[j]) , np.nan , y_vals[j])
            f2.plot(axsmer, y, label=y_labs[j], color = c[j])
            f = f2

        f.xaxis.set_major_formatter(formatter)
        f.set_xlim(datemin,datemax)
        f.legend(loc='upper center',ncol=3, bbox_to_anchor=[0.5, 1.4])
        f.grid()
        ax[2].legend(bbox_to_anchor=[0.25, 1.4])
        ax3 = ax[2].twinx()
        for k in [0,1]:
            pc_power, power_times = pn.ulf_power(k)
#             print(type(pc_power), len(pc_power['mean_pc_power']))
#             print(len(power_times))
            ax3.plot(power_times, pc_power['mean_pc_power'], label= f'pc{k+2} mean power')
            ax3.xaxis.set_major_formatter(formatter)
        ax3.legend(loc='upper center', ncol=2, bbox_to_anchor=[0.5, 1.4])
        ax3.set_xlim(datemin,datemax)
        ax3.set_ylabel('log(nT)^2')
        f.set_xticklabels([])
        f2.set_ylabel('nT')
        f2.minorticks_on()
        f2.set_xlim(datemin,datemax)
        f2.set_xticklabels([])

    for band in [0,1]:
        plot_num = 3+band
        ylims = dict()
        ylims[band]=[]
        plt.text(0.1, 0.8,f'all_networks_pc{band+2}',transform=ax[plot_num].transAxes, bbox = dict(facecolor = 'white', alpha = 0.5))
        ax[plot_num].grid(visible=True)        
        for comp in ['n','e','z']:
            vals = num_conn[band][comp]
            ylims[band].append(max(vals['total']))
            t = pd.to_datetime(vals.index)
            ax[plot_num].plot(t ,vals['total'].values, label=f'{comp}')
            ax[plot_num].legend(loc='upper right')
            ax[plot_num].xaxis.set_major_formatter(formatter)
            ax[plot_num].set_xlim(datemin,datemax)
        ax[plot_num].set_ylabel('# conn.')
    ax[3].axes.get_xaxis().set_visible(False)
    plt.savefig(f'plots/{year}_nets/combined_nets_all_comps_{year}_{handel}.png', facecolor='w')
    plt.show()

In [ ]:
plot_edge_num_ts1(num_conn, indices_fname)

In [ ]:
def plot_edge_num_ts2(num_conn:dict, indices_fname:str):
    plt.rcParams.update({'font.size': 15})
    num_plots = 7
    fig, ax = plt.subplots(num_plots, figsize=(15, 10))
    fig.suptitle('spd')
    indices = pd.read_csv(indices_fname)
    index_date = indices['Date_UTC']
    sme = indices['SME']
    axsmer = pd.to_datetime(index_date)
    datemin = datetime.strptime(pn.dt_start, '%Y-%m-%d %H:%M:%S')
    datemax = datetime.strptime(pn.dt_end, '%Y-%m-%d %H:%M:%S')
    f1 = ax[0]
    f1.plot(axsmer,sme, color='black', label = 'SME')
    f1.set_ylabel('SME (nT)')
    f1.grid()
#   turn off labels for x axis while preserving ticks
    f1.set_xticklabels([])                           
    f1.set_xlim(datemin,datemax)
    formatter = mdates.DateFormatter("%H:%M")
    f1.xaxis.set_major_formatter(formatter)
    f1.legend()
    f1.set_xlim(datemin,datemax)
    ax2 = f1.twinx()

    # code for plotting SMR sections for entire SMR yearly dataset
    indices['Date_UTC'] = pd.to_datetime(indices['Date_UTC']) 
    for i in ['SMR00','SMR06','SMR12','SMR18' ]:
        x = axsmer[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)]
        y = indices[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)][i]
        ax2.plot(x , y, label= i)
    ax2.xaxis.set_major_formatter(formatter)
    ax2.legend(loc='upper center',ncol=4, bbox_to_anchor=[0.5, 1.5])
    ax2.set_ylabel('SMR (nT)')
    ax2.grid()
    ax2.set_xlim(datemin,datemax)
    ax2.set_xticklabels([])
    step_arr=[0,3]
    for band in [0,1]:
        for comp_ind, comp in enumerate(['n','e','z']):
            plot_num = step_arr[band] + comp_ind+1
            print(f'plot_num is {plot_num}', band, comp)
            plt.text(0, 0.6,f'pc{band+2}, {comp}',transform=ax[plot_num].transAxes, bbox = dict(facecolor = 'white', alpha = 0.2))
            for net_label in net_labs:
                vals = num_conn[band][comp][net_label]
                t = pd.to_datetime(vals.index)
                ax[plot_num].plot(t ,vals['total'].values, label=f'{net_label}')
                ax[plot_num].set_xlim(datemin,datemax)
            ax[plot_num].grid()
            ax[plot_num].legend(loc='upper center',ncol=4, bbox_to_anchor=[0.5, 1.4])
            ax[plot_num].set_ylabel('# conn.')
            if plot_num<6:
                ax[plot_num].set_xticklabels([])

        ax[6].xaxis.set_major_formatter(formatter)
        ax[6].set_xlabel('time hr:min')
        ax[1].set_xticklabels([])
        plt.savefig(f'plots/{year}_nets/all_networks_all_comps_{handel}.png', facecolor='w')
    plt.show()

In [ ]:
num_conn2 = pn.all_nets_conn()

In [ ]:
plot_edge_num_ts2(num_conn2, indices_fname)

In [ ]:
def plot_short_long_ts(indices_fname:str, num_conn:dict, band:int, comp:str):
    plt.rcParams.update({'font.size': 15})
    num_plots = 1 + len(net_labs)
    fig, ax = plt.subplots(num_plots, figsize=(12, 8))
    fig.suptitle(f'Pc{band+2}, comp. {comp}')
    indices = pd.read_csv(indices_fname)
    index_date = indices['Date_UTC']
    sme = indices['SME']
    axsmer = pd.to_datetime(index_date)
    datemin = datetime.strptime(dt_start, '%Y-%m-%d %H:%M:%S')
    datemax = datetime.strptime(dt_end, '%Y-%m-%d %H:%M:%S')
    f1 = ax[0]
    f1.plot(axsmer,sme, color='black', label = 'SME')
    f1.set_ylabel('SME (nT)')
    f1.grid()
#   turn off labels for x axis while preserving ticks
    f1.set_xticklabels([])                           
    f1.set_xlim(datemin,datemax)
    formatter = mdates.DateFormatter("%H:%M")
    f1.xaxis.set_major_formatter(formatter)
    f1.legend()
    f1.set_xlim(datemin,datemax)
    ax2 = f1.twinx()

    # code for plotting SMR sections for entire SMR yearly dataset
    indices['Date_UTC'] = pd.to_datetime(indices['Date_UTC']) 
    for i in ['SMR00','SMR06','SMR12','SMR18' ]:
        x = axsmer[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)]
        y = indices[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)][i]
        ax2.plot(x , y, label= i)
    ax2.xaxis.set_major_formatter(formatter)
    # ax2.legend(loc=10)
    ax2.set_ylabel('SMR (nT)')
    ax2.grid()
    ax2.set_xlim(datemin,datemax)
    ax2.set_xticklabels([])

    for ind, val in enumerate(net_labs):
        plot_num = ind+1
        plt.text(0.5, 1,f'{val}', ha='center', va='center', transform=ax[plot_num].transAxes, bbox = dict(facecolor = 'white', alpha = 0.5))
        short_df = num_conn[val]['short']
        long_df = num_conn[val]['long']
        t = pd.to_datetime(short_df.index)
        ax[plot_num].plot(t ,short_df['short'].values, color= 'blue', label = 'short', lw=3)
        ax[plot_num].plot(t ,long_df['long'].values, color= 'red', label = 'long', linestyle='--')
        ax[plot_num].legend(loc='upper left', ncol=2, bbox_to_anchor=[0, 1.3] )
        ax[plot_num].xaxis.set_major_formatter(formatter)
        ax[plot_num].set_xlim(datemin,datemax)
        ax[plot_num].set_ylabel('# conn.')
        ax[plot_num].grid()

        if plot_num<len(net_labs):
            ax[plot_num].set_xticklabels([])
        plt.savefig(f'plots/{year}_nets/comp_{comp}/long_short_all_nets_{comp}_pc{band+2}_{handel}', facecolor='w')
    

In [ ]:
for band in [0,1]:
    for comp in ['e','n','z']:
        num_conn3 = pn.all_nets_short_long_conn(band=band, comp=comp)
        plot_short_long_ts(indices_fname, num_conn3, band=band, comp=comp)

  1%|          | 3503/600067 [00:00<00:17, 35026.19it/s]

In [ ]:
def plot_edge_num_ts4(net_label:str, num_conn:dict, indices_fname:str):
    plt.rcParams.update({'font.size': 15})
    num_plots = 7
    fig, ax = plt.subplots(num_plots, figsize=(15, 10))
    fig.suptitle(f'spd, {net_label}')
    indices = pd.read_csv(indices_fname)
    index_date = indices['Date_UTC']
    sme = indices['SME']
    axsmer = pd.to_datetime(index_date)
    datemin = datetime.strptime(pn.dt_start, '%Y-%m-%d %H:%M:%S')
    datemax = datetime.strptime(pn.dt_end, '%Y-%m-%d %H:%M:%S')
    f1 = ax[0]
    f1.plot(axsmer,sme, color='black', label = 'SME')
    f1.set_ylabel('SME (nT)')
    f1.grid()
#   turn off labels for x axis while preserving ticks
    f1.set_xticklabels([])                           
    f1.set_xlim(datemin,datemax)
    formatter = mdates.DateFormatter("%H:%M")
    f1.xaxis.set_major_formatter(formatter)
    f1.legend()
    f1.set_xlim(datemin,datemax)
    ax2 = f1.twinx()

    # code for plotting SMR sections for entire SMR yearly dataset
    indices['Date_UTC'] = pd.to_datetime(indices['Date_UTC']) 
    for i in ['SMR00','SMR06','SMR12','SMR18' ]:
        x = axsmer[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)]
        y = indices[(indices['Date_UTC'] >= datemin) & (indices['Date_UTC'] <= datemax)][i]
        ax2.plot(x , y, label= i)
    ax2.xaxis.set_major_formatter(formatter)
    ax2.legend(loc='upper center',ncol=4, bbox_to_anchor=[0.5, 1.5])
    ax2.set_ylabel('SMR (nT)')
    ax2.grid()
    ax2.set_xlim(datemin,datemax)
    ax2.set_xticklabels([])
    step_arr=[0,3]
    for band in [0,1]:
        for comp_ind, comp in enumerate(['n','e','z']):
            plot_num = step_arr[band] + comp_ind+1
            plt.text(0, 0.6,f'pc{band+2}, {comp}',transform=ax[plot_num].transAxes, bbox = dict(facecolor = 'white', alpha = 0.2))
            short_df = num_conn[band][comp]['short']
            long_df = num_conn[band][comp]['long']
            t = pd.to_datetime(short_df.index)
            ax[plot_num].plot(t ,short_df['short'].values, color= 'blue', label = f'short pc{band+2}, {comp}', lw=3)
            ax[plot_num].plot(t ,long_df['long'].values, color= 'red', label = f'long pc{band+2}, {comp}', linestyle='--')
            ax[plot_num].set_xlim(datemin,datemax)
            ax[plot_num].grid()
            ax[plot_num].legend(loc='upper center',ncol=2, bbox_to_anchor=[0.5, 1.4])
            ax[plot_num].set_ylabel('# conn.')
            if plot_num<6:
                ax[plot_num].set_xticklabels([])

        ax[6].xaxis.set_major_formatter(formatter)
        ax[6].set_xlabel('time hr:min')
        ax[1].set_xticklabels([])

        
    plt.savefig(f'plots/{year}_nets/{net}_all_comps_long_short_{handel}.png', facecolor='w')
    plt.show()

In [ ]:
for net in net_labs:
    net_conn4 = pn.single_net_all_comp_bands(net)
    plot_edge_num_ts4(net, net_conn4, indices_fname)
